# Python Inventory System - Part 4: CLI Interface

This notebook covers the Command Line Interface (CLI) for our Inventory Management System, focusing on the `cli.py` file. We'll learn how to build an interactive command-line application using Typer and Rich for beautiful console output.

## What We'll Cover

1. Introduction to Typer and Rich
2. CLI Command Structure
3. Implementing CRUD Operations
4. Displaying Data with Rich Tables
5. Error Handling and User Feedback

## 1. Introduction to Typer and Rich

**Typer** is a library for building CLI applications that's intuitive and easy to use. It's built on top of Click but uses Python type hints for defining commands and options.

**Rich** is a Python library for rich text and beautiful formatting in the terminal.

Let's install these dependencies first:

In [ ]:
!pip install typer rich

## 2. CLI Command Structure

Our CLI application has several commands organized under a main Typer app:

In [ ]:
import typer

# Create the main Typer app
app = typer.Typer(help="Inventory Management System")

# Example command structure
@app.command("init")
def initialize_database():
    """Initialize the database with tables"""
    print("Database initialized successfully!")

# This would be called with: python cli.py init
# Try running this cell and then the command above

### Key Components:

1. **`@app.command()`**: Decorator to register a function as a CLI command
2. **Command Help Text**: The function docstring becomes the command's help text
3. **Options**: Defined using `typer.Option()` with various parameters
4. **Rich Console**: Used for colored and formatted output

## 3. Implementing CRUD Operations

Our CLI implements four main CRUD operations:

1. **Add** - Create new products
2. **Edit** - Update existing products
3. **Delete** - Remove products
4. **List** - View all products

### 3.1. Adding Products

The `add` command creates new products with all required fields:

In [ ]:
from rich.console import Console
from models import Product

console = Console()

@app.command("add")
def add_product_command(
    name: str = typer.Option(..., prompt=True, help="Product name"),
    description: str = typer.Option(..., prompt=True, help="Product description"),
    price: float = typer.Option(..., prompt=True, help="Product price"),
    stock_quantity: int = typer.Option(..., prompt=True, help="Current stock quantity"),
    category: str = typer.Option(..., prompt=True, help="Product category")
):
    """Add a new product to inventory"""
    # In a real implementation, this would call add_product() from operations.py
    product = Product(
        name=name,
        description=description,
        price=price,
        stock_quantity=stock_quantity,
        category=category
    )
    
    console.print(f"[green]Product '{product.name}' added successfully![/green]")
    console.print(f"[blue]Details: {product}[/blue]")

# Example usage:
# python cli.py add --name "Laptop" --description "Powerful laptop" --price 999.99 --stock-quantity 10 --category "Electronics"

### Key Features:

- `typer.Option(..., prompt=True)`: Makes the option required and prompts if not provided
- Type hints (`str`, `float`, `int`) ensure proper data types
- Rich formatting with `[green]` and `[/green]` for colored output

### 3.2. Editing Products

The `edit` command updates existing products with any provided fields:

In [ ]:
@app.command("edit")
def edit_product_command(
    product_id: int = typer.Option(..., prompt=True, help="Product ID to edit"),
    name: Optional[str] = typer.Option(None, help="New product name"),
    description: Optional[str] = typer.Option(None, help="New product description"),
    price: Optional[float] = typer.Option(None, help="New product price"),
    stock_quantity: Optional[int] = typer.Option(None, help="New stock quantity"),
    category: Optional[str] = typer.Option(None, help="New product category")
):
    """Edit an existing product"""
    # In a real implementation, this would call edit_product() from operations.py
    changes = {
        "name": name,
        "description": description,
        "price": price,
        "stock_quantity": stock_quantity,
        "category": category
    }
    
    # Filter out None values (unchanged fields)
    changes = {k: v for k, v in changes.items() if v is not None}
    
    if changes:
        console.print(f"[yellow]Updating product ID {product_id} with: {changes}[/yellow]")
        console.print(f"[green]Product (ID: {product_id}) updated successfully![/green]")
    else:
        console.print("[yellow]No changes provided, nothing to update.[/yellow]")

# Example usage:
# python cli.py edit --product-id 1 --price 899.99 --stock-quantity 8

### Key Features:

- Optional parameters with `Optional[str]` and `typer.Option(None)`
- Only updates fields that are provided
- Clear feedback about what's being updated

### 3.3. Deleting Products

The `delete` command removes products by their ID:

In [ ]:
@app.command("delete")
def delete_product_command(
    product_id: int = typer.Option(..., prompt=True, help="Product ID to delete")
):
    """Delete a product from inventory"""
    # In a real implementation, this would call delete_product() from operations.py
    console.print(f"[red]Deleting product with ID {product_id}[/red]")
    console.print(f"[green]Product (ID: {product_id}) deleted successfully![/green]")

# Example usage:
# python cli.py delete --product-id 1

## 4. Displaying Data with Rich Tables

The `list` command shows all products in a beautifully formatted table:

In [ ]:
from rich.table import Table

@app.command("list")
def list_products_command():
    """List all products in inventory"""
    # Sample data - in real implementation this would come from list_products()
    products = [
        Product(id=1, name="Laptop", description="Powerful laptop", price=999.99, stock_quantity=10, category="Electronics"),
        Product(id=2, name="Mouse", description="Wireless mouse", price=29.99, stock_quantity=25, category="Accessories"),
        Product(id=3, name="Keyboard", description="Mechanical keyboard", price=89.99, stock_quantity=15, category="Accessories")
    ]
    
    # Create a Rich Table
    table = Table(show_header=True, header_style="bold blue")
    table.add_column("ID", style="dim")
    table.add_column("Name")
    table.add_column("Description")
    table.add_column("Price", justify="right")
    table.add_column("Stock", justify="right")
    table.add_column("Category")
    
    # Add rows to the table
    for product in products:
        table.add_row(
            str(product.id),
            product.name,
            product.description,
            f"${product.price:.2f}",
            str(product.stock_quantity),
            product.category
        )
    
    console.print(table)

# Display the sample table
list_products_command()

### Key Features:

- Beautiful table formatting with headers and styles
- Column alignment (right for numbers)
- Consistent formatting of prices with 2 decimal places
- Clear visual hierarchy

## 5. Error Handling and User Feedback

The CLI provides clear feedback for both success and error cases:

In [ ]:
# Example of success and error messages
console.print("[green]Operation completed successfully![/green]")
console.print("[yellow]Warning: Low stock inventory[/yellow]")
console.print("[red]Error: Product not found[/red]")

# Example of handling a not-found case
product_id = 999
found = False  # Simulate not found

if found:
    console.print(f"[green]Product (ID: {product_id}) updated successfully![/green]")
else:
    console.print(f"[red]Product with ID {product_id} not found![/red]")

## Summary

In this notebook, we've learned:

1. **Typer basics** for building CLI applications
2. **Rich formatting** for beautiful console output
3. **CRUD command implementation** for product management
4. **Data display** with Rich tables
5. **User feedback** with colored status messages

This CLI interface provides a user-friendly way to interact with our inventory system. The commands are intuitive and provide clear feedback, making the system easy to use for inventory management tasks.